In [ ]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os
%load_ext dotenv
%dotenv

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Build Docker image

In [ ]:
docker_login = "<your-docker-login>"
aks_image_repo = "batchscoringdl_scoring_app"

In [ ]:
%%writefile dequeuer_app/requirements.txt
azure==4.0.0
torch==0.4.1
torchvision==0.2.1

In [ ]:
%%writefile dequeuer_app/Dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        curl \
        git \
        nginx \
        supervisor \
        wget && \
        rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.6
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION && \
    /opt/conda/bin/conda clean -ya
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
ENV PYTHONPATH /code/:$PYTHONPATH

RUN mkdir /app
WORKDIR /app
ADD process_images_from_queue.py /app
ADD style_transfer.py /app
ADD requirements.txt /app

RUN pip install --no-cache-dir -r requirements.txt

CMD ["python", "process_images_from_queue.py"]

In [ ]:
!sudo docker build -t $dequeuer_image_repo dequeuer_app

In [ ]:
set_key(env_path, "DOCKER_LOGIN", docker_login)
set_key(env_path, "DEQUEUER_IMAGE_REPO", image_repo)

## Add new items to queue

In [ ]:
!python queuer_app/add_images_to_queue.py --input-dir "input" --queue-limit 3

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"

## Dequeue using docker image

In [ ]:
!sed -e "s/=\"/=/g" -e "s/\"$//g" .env > .env.docker

In [ ]:
!cat .env.docker

In [ ]:
!sudo docker run --runtime=nvidia --env-file ".env.docker" $image_repo

In [ ]:
!rm .env.docker

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query 'countDetails.activeMessageCount'

In [ ]:
!sudo docker tag $image_repo $docker_login/$dequeuer_image_repo

In [ ]:
!sudo docker push $docker_login/$dequeuer_image_repo

---

## Conclusion

In [ ]:
!cat .env